In [1]:
import numpy as np
import pandas as pd
import glob
import os.path as op
from os import makedirs
import shutil

def convert_parfile(infile, outfile, duration=18):
    parfile = pd.read_table(infile, header=None, names=['onset','blocktype','trial_type','color'])
    assert(len(np.unique(parfile.blocktype))==3) # we only expect M, P, and blank blocks.
    parfile['duration'] = duration # default 18 (3T), 16 for 7T
    out = parfile[['onset','duration','trial_type']]
    out.to_csv(outfile,"\t", index=False)

def convert_parfiles(in_glob, out_stem):
    parfiles = glob.glob(in_glob)
    tsvfiles = [out_stem + op.basename(p).split('run')[-1].split('.')[0] + '_events.tsv' for p in parfiles]
    for (par, tsv) in zip(parfiles, tsvfiles):
        convert_parfile(par, tsv)
    

In [ ]:
#convert 20150401
convert_parfiles("/Users/smerdis/data/LGN/MS_LGN/20150401MS.dicom/20150401MS/Stimuli/parfiles/MS_20150401_run*.par",
                 "/Users/smerdis/data/LGN/BIDS/AlexLGN_no0327/sub-MS/ses-20150401/func/sub-MS_ses-20150401_task-mp_run-")

In [ ]:
convert_parfiles("/Users/smerdis/data/LGN/MS_LGN/20150519MS.dicom/20150519MS/Stimuli/parfiles/MS_20150519_run*.par",
                 "/Users/smerdis/data/LGN/BIDS/AlexLGN_no0327/sub-MS/ses-20150519/func/sub-MS_ses-20150519_task-mp_run-")

In [ ]:
def migrate_alex2015():
    sessions_to_group = {#'20150327':'/Users/smerdis/data/LGN/MS_LGN/20150327MS.dicom/20150327MS/20150327MS_dicom/',
    '20150401':'/Users/smerdis/data/LGN/MS_LGN/20150401MS.dicom/20150401MS/20150401MS_backup/',
    '20150519':'/Users/smerdis/data/LGN/MS_LGN/20150519MS.dicom/20150519MS/20150519MS_dicom/'}

    dataset_name = 'AlexLGN_no0327'
    sub_code = 'MS'
    staging_folder = op.join('/Users/smerdis/data/LGN', dataset_name, sub_code)
    if not op.exists(staging_folder):
        makedirs(staging_folder)
        
    print(sessions_to_group)
    
    for k,v in sessions_to_group.items():
        session_folder = op.join(staging_folder, k)
        if not op.exists(session_folder):
            makedirs(session_folder)
        dicoms = glob.glob((v + "epi*/IM-*.dcm"))
        for f in dicoms:
            shutil.copy(f, session_folder) #put all dicoms from that day in one folder


In [ ]:
migrate_alex2015()

In [ ]:
['L', 'R'] * 6

In [34]:
def write_hemifield_localizer_event_file(event_file):
    num_LR_cycles = 11
    len_hemifield_cycle = 13.5
    TR = 2.25

    #total_stim_length = (len_hemifield_cycle * 2) * num_LR_cycles
    #total_volumes = total_stim_length/TR
    volumes_per_hemicycle = len_hemifield_cycle/TR

    conds = ['R', 'L'] # we start with L, actually
    #print(total_stim_length, volumes_per_hemicycle, total_volumes)

    file_contents = "onset\tduration\ttrial_type\n"
    for hemicycle in range(1, num_LR_cycles*2+1):
        file_contents += f"{hemicycle*len_hemifield_cycle}\t{len_hemifield_cycle}\t{conds[hemicycle%len(conds)]}\n"

    with open(event_file, 'w') as f:
        f.write(file_contents)

In [35]:
from bids.grabbids import BIDSLayout
raw_layout = BIDSLayout("/Users/smerdis/data/LGN/BIDS/AlexLGN_no0327/")
hemifield_event_files = raw_layout.get(subject='MS', modality='func', 
                                       task="hemi", extensions=['.tsv'])
for f in hemifield_event_files:
    write_hemifield_localizer_event_file(f.filename)

In [29]:
list(range(1, num_LR_cycles*2+1))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

In [14]:
%ls

Event file conversion.ipynb
LGN_MP_Connectivity_Analyses.xlsx
LGN_MP_Functional_Connectivity_20140407.key
LGN_MP_Spatial_Analyses2_20140407.key
LGN_MP_Spatial_Analyses_20140407.key
Pipeline_AS/
Porcupine/
Untitled.ipynb
Untitled1.ipynb
build-Porcupine-Desktop_Qt_5_10_0_clang_64bit-Release/
build-Porcupine-Desktop_Qt_5_9_3_clang_64bit-Release/
figures/
glm2/
glm_pipeline/
megavista/
pycortex-gallantlab/


In [10]:
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.algorithms.modelgen as model   # model generation

from numpy import array

In [9]:
np.array?

In [2]:
spm.Level1Design?

In [11]:
session_info = [{'cond': [{'amplitudes': array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]), 'duration': [13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5], 'name': 'L', 'onset': [13.5, 40.5, 67.5, 94.5, 121.5, 148.5, 175.5, 202.5, 229.5, 256.5, 283.5]}, {'amplitudes': array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]), 'duration': [13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5], 'name': 'R', 'onset': [27.0, 54.0, 81.0, 108.0, 135.0, 162.0, 189.0, 216.0, 243.0, 270.0, 297.0]}], 'hpf': 120.0, 'regress': [{'name': 'Realign1', 'val': [0.0, -0.0063602568, -0.0092585091, -0.015699494, -0.021592326, -0.031327726, -0.026519606, -0.031310321, -0.021006917, -0.031746479, -0.031656015, -0.030933885, -0.027607314, -0.037002105, -0.030493413, -0.0302945, -0.023375407, -0.025998008, -0.018474344, -0.028056897, -0.024511251, -0.027236357, -0.033701428, -0.03601059, -0.026959825, -0.019386965, -0.02820016, -0.024476112, -0.037758022, -0.031569943, -0.031736109, -0.028973323, -0.028016971, -0.034266357, -0.031285008, -0.039612867, -0.048353658, -0.050739149, -0.039068118, -0.036664219, -0.039885852, -0.035803574, -0.0410139, -0.037275274, -0.039841485, -0.03527734, -0.034610045, -0.032603847, -0.031265827, -0.035383008, -0.035686812, -0.034975469, -0.032336399, -0.03008724, -0.033474408, -0.027300016, -0.021604062, -0.02620657, -0.034652423, -0.045616629, -0.045123588, -0.047022718, -0.04681534, -0.042799893, -0.041904159, -0.033172688, -0.041953393, -0.047725641, -0.043331286, -0.053341574, -0.046128723, -0.051991451, -0.051019419, -0.05277802, -0.042557452, -0.038474743, -0.03205317, -0.043121536, -0.042553231, -0.044907874, -0.047958263, -0.040266161, -0.039865083, -0.036625207, -0.041741873, -0.045322088, -0.043800066, -0.044423752, -0.039935871, -0.039211087, -0.042291005, -0.043452146, -0.039707791, -0.047098023, -0.045674227, -0.046811898, -0.062183716, -0.066438095, -0.059778745, -0.049748136, -0.048341978, -0.050272593, -0.048549205, -0.050913018, -0.054408479, -0.058773536, -0.05658172, -0.04802466, -0.04681701, -0.045548296, -0.042150915, -0.042026378, -0.034785801, -0.036977673, -0.035676423, -0.034985184, -0.035774983, -0.040641596, -0.048374302, -0.064717447, -0.063140543, -0.064629243, -0.053917777, -0.05268991, -0.049764165, -0.055956915, -0.052490898, -0.047360334, -0.064761595, -0.067820378, -0.071430157, -0.069937449, -0.064095532, -0.070439771, -0.052970605, -0.049202888, -0.051808323, -0.049411056, -0.04067058]}, {'name': 'Realign2', 'val': [-1.113348e-14, 0.079471728, 0.40905198, -0.20319787, -0.27914987, 0.051458391, -0.091976653, 0.14113167, 0.048094161, 0.25311987, -0.038550369, 0.1765388, -0.014692671, 0.23268089, 0.011428187, 0.09613765, 0.1574803, -0.11887675, 0.20913174, 0.073925543, 0.55080779, 0.03335285, -0.15703593, 0.14438735, 0.043928157, -0.11197714, -0.034720396, -0.052351025, -0.0036467607, 0.24345718, -0.18321719, 0.23413485, -0.062436564, 0.17356319, 0.42376923, -0.044619119, -0.082120153, -0.1656488, -0.10175298, -0.014259977, 0.052779432, 0.058180356, 0.30736559, -0.087574453, 0.25823445, -0.041817939, -0.10263285, 0.19973301, -0.12760134, 0.09922253, 0.098222792, -0.16016266, 0.24084392, -0.17661663, 0.17562514, 0.030966382, -0.010522835, -0.18096417, -0.23171861, 0.29674795, 0.22765362, -0.23080877, -0.0056586836, -0.010045901, -0.063632893, 0.019115133, -0.16496905, 0.049296553, 0.018690088, 0.34688292, 0.33340507, -0.082861083, -0.059686173, -0.1879205, -0.22429349, -0.11127739, -0.18466933, 0.05913232, 0.3111223, -0.14023763, 0.33098919, -0.20968384, 0.08365232, 0.1462722, -0.0099215292, 0.37364971, -0.16114422, 0.16227381, -0.055697707, 0.17606364, -0.0474132, 0.19200599, 0.24846926, -0.14705264, 0.24341464, -0.0051642047, 0.26975158, 0.08644606, 0.059029078, 0.13752315, -0.14448712, 0.25817547, 0.10128999, 0.29338942, 0.020715556, 0.36884981, -0.038925052, 0.41524577, -0.019831168, 0.21024612, 0.12990198, 0.33697346, 0.11081356, 0.37526119, 0.0053608041, 0.4855493, -0.0041533093, -0.055095905, -0.041104722, 0.26632634, -0.030160786, 0.35627357, 0.016795145, 0.16893322, 0.16462797, 0.26592888, 0.36419429, -0.063528247, 0.040292642, 0.18296629, -0.079414007, 0.24760191, -0.096585837, 0.26564866, -0.0065049222, 0.19398262, 0.22875144, -0.16371444, 0.32104644]}, {'name': 'Realign3', 'val': [0.0, 0.05465287, -0.021313245, -0.11118867, -0.026362122, -0.015175999, -0.03601989, 0.012551804, -0.10973633, -0.075921882, -0.1477182, -0.068597746, -0.1264585, -0.022688057, -0.099407621, -0.079325674, -0.10907008, -0.054011415, -0.063799631, -0.10103318, -0.16804348, -0.23301189, -0.20545565, -0.1278365, -0.20904664, -0.14804427, -0.10947357, -0.18255591, -0.071067506, -0.087029977, -0.10240949, -0.067771627, -0.14634739, -0.14179929, -0.091305293, -0.12637683, -0.15977191, -0.083708486, -0.17409907, -0.16317618, -0.1176734, -0.16330325, -0.072080557, -0.04641267, -0.04413872, -0.16234743, -0.081953945, -0.089313891, -0.12176639, -0.044558593, 0.0099695835, -0.073883345, -0.099718628, -0.072457072, -0.0031662714, -0.066356037, -0.2058478, -0.15071334, -0.11254141, -0.067178122, -0.10891273, -0.13799583, -0.031374708, -0.048697831, -0.018485204, -0.09592931, -0.011099181, 0.040826509, -0.082216759, -0.17256882, -0.26121194, -0.25891786, -0.31326068, -0.3345513, -0.34847982, -0.29410476, -0.32585566, -0.21015546, -0.22190813, -0.12802648, -0.16781968, -0.28571713, -0.23479883, -0.28670918, -0.2109332, -0.17487037, -0.19995981, -0.23914997, -0.27721322, -0.20166541, -0.22968459, -0.2196106, -0.30696183, -0.26649147, -0.22248319, -0.26062601, -0.31267861, -0.37195913, -0.22836031, -0.2678129, -0.35463371, -0.27439749, -0.23218589, -0.23862632, -0.25967977, -0.27682368, -0.3610956, -0.47915547, -0.38544532, -0.34115004, -0.4270464, -0.42401911, -0.40375969, -0.36346409, -0.40149235, -0.44651986, -0.46260315, -0.40689743, -0.342632, -0.3004415, -0.3455679, -0.34222107, -0.30385891, -0.31789374, -0.44983203, -0.3839021, -0.40439028, -0.42845718, -0.36930376, -0.45640567, -0.40004514, -0.37408513, -0.34489103, -0.27914423, -0.28400418, -0.049628004, -0.10457497, -0.078391066, -0.096701839]}, {'name': 'Realign4', 'val': [0.0, 7.3119938e-05, -0.0024757374, -0.0025942092, -0.0020327365, -0.0021301159, -0.0012040514, -0.00090354332, -0.0015848745, -0.0015038115, -0.0019781715, -0.001146324, -0.0019943791, -0.0010107589, -0.001032589, -0.00051657873, -0.0012555693, 0.00064565668, -0.0003466692, -0.0013711848, -0.0024051651, -0.0025484355, -0.0018220034, -0.0012385153, -0.0034848838, -0.0033556514, -0.0033198165, -0.0037273296, -0.0018872432, -0.0019695801, -0.0015889474, -0.00099929487, -0.0029188237, -0.0027561714, -0.0034725917, -0.003623683, -0.003279042, -0.0019255248, -0.0024609128, -0.0028663464, -0.0022916212, -0.0015271138, -0.0013850213, -0.0016788131, -0.0014851418, -0.0022183817, -0.0021660348, -0.0020870466, -0.0014291513, -0.0016077046, -0.0018637242, -0.0017359244, -0.0022540117, -0.0019223012, -0.0014125359, -0.0019915723, -0.0043798918, -0.0044738955, -0.0043237774, -0.0028477095, -0.003539, -0.0023008192, -0.0013658334, -0.0021003334, -0.0016111669, -0.002550455, -0.0029044674, -0.0018833466, -0.0024517008, -0.0022512414, -0.0047341363, -0.0048384852, -0.0062473973, -0.0057688029, -0.0059510449, -0.0065729704, -0.006244268, -0.0044504029, -0.0042297722, -0.0032792213, -0.0037421707, -0.0044786681, -0.0039960214, -0.0045087786, -0.0034786371, -0.0037872411, -0.0033074323, -0.0030926846, -0.0032563881, -0.0037653809, -0.0043540461, -0.0037551101, -0.0048676337, -0.0039598272, -0.0038523703, -0.004370628, -0.0037682918, -0.0051668223, -0.0025129921, -0.0036349495, -0.0039192707, -0.0038696182, -0.0037573953, -0.0039238769, -0.0035718526, -0.0041738799, -0.0039888464, -0.0059315154, -0.0047056473, -0.0042147496, -0.0046838812, -0.0044246092, -0.0039493572, -0.0038461281, -0.0033044829, -0.0066552693, -0.0063320499, -0.0067271284, -0.0059097014, -0.0042488767, -0.0039974239, -0.0039016591, -0.0037886312, -0.0035159309, -0.0043459744, -0.0029635443, -0.0037380359, -0.0067770268, -0.0046138901, -0.006365368, -0.0056521164, -0.0049097545, -0.0040868664, -0.0031215214, -0.0023718758, -0.0017481268, -0.0024097475, -0.00068718041, -0.0022244486]}, {'name': 'Realign5', 'val': [0.0, -0.00035397837, -0.0008791832, -0.00084456979, -0.0018185399, -0.0016853094, -0.0020149631, -0.0023356144, -0.0016987142, -0.0016056509, -0.000997455, -0.0012581552, -0.00099979813, -0.0017880785, -0.0016697991, -0.0016576534, -0.0018644337, -0.0011002847, -0.0010716308, -0.00076351251, -0.0015615442, -0.0014250361, -0.0012208268, -0.0013974075, -0.0007766288, -0.00078853331, -0.0018264524, -0.0017857668, -0.0015305925, -0.0018469898, -0.0018356061, -0.0014459329, -0.0017270997, -0.0018001689, -0.0018116207, -0.0019489799, -0.0016182921, -0.0021460978, -0.0019000699, -0.001974993, -0.0021593193, -0.0024601298, -0.0030479633, -0.0026966376, -0.0026385935, -0.0021941056, -0.0021242855, -0.0020927782, -0.0016908688, -0.0015815681, -0.0017274819, -0.0017088903, -0.0016016222, -0.0015664655, -0.0016195385, -0.0010490755, 0.00038883468, -0.0011772751, -0.0010896812, -0.0013225036, -0.0012662564, -0.0012720142, -0.0014796501, -0.001483542, -0.0013910844, -0.0012831296, -0.0021462611, -0.0021368346, -0.0015818803, -0.00137837, -0.0012133752, -0.0014173555, -0.001620739, -0.0011762612, -0.00048275273, -0.0012038064, -0.0014667951, -0.0015796644, -0.0027231555, -0.0027029403, -0.0021618771, -0.0015592704, -0.0016109414, -0.0021149941, -0.0019390162, -0.0010931422, -0.00058962804, -0.00057791368, -0.00059523521, -0.00093017863, -0.0011396855, -0.00085715452, -0.00082949563, -0.00049909722, -0.00062786305, -0.00044072178, -0.00097232303, -0.0019749051, -0.0016398581, -0.0019719819, -0.0021019231, -0.002425838, -0.0024773463, -0.002205492, -0.0019986862, -0.0015936338, -0.0012341017, -0.00089101647, -0.0013478171, -0.0014644328, -0.0013516316, -0.0010431469, -0.0011160783, -0.00084946316, -0.0012574417, -0.0010439422, -0.00056952737, -0.00096757001, -0.00098586214, -0.0011426183, -0.0013561603, -0.0010555263, -0.0013315676, -0.0015181797, -0.0014897766, -0.0015158122, -0.0011949798, -0.00017419291, -0.0010156471, -0.0010168424, -0.00094361752, -0.0010398587, -0.00096680446, -0.00017034845, -0.00045346938, -0.00098033652, -0.0011317076, -0.0010013313, -0.0011057508]}, {'name': 'Realign6', 'val': [-2.220439e-32, 3.2104776e-05, -0.00015773123, -0.00012366347, -4.4190572e-05, -0.00028968165, -0.00031854791, -0.00015682533, -8.2377057e-05, -1.545117e-05, -9.9392053e-05, -7.7845862e-05, -0.00022268744, -0.00036956864, -0.00065952366, -0.00069251866, -0.00089474789, -0.00067986777, -0.00070335054, -0.00074100684, -0.00077483372, -0.00062052124, -0.00057150753, -0.00054606394, -0.000576354, -0.00055737026, -0.00091685209, -0.0012150494, -0.0011301485, -0.0013209944, -0.0013452956, -0.0011038103, -0.001290947, -0.0011420194, -0.0010667366, -0.0010642009, -0.0010683813, -0.0011590084, -0.001416067, -0.0014345751, -0.0013488889, -0.001346369, -0.0011648619, -0.00098885473, -0.000880381, -0.00093390543, -0.00079854943, -0.00090422532, -0.00068854083, -0.00090546993, -0.0010211833, -0.0010681889, -0.0012638012, -0.0013234252, -0.0011610914, -0.00079216154, -0.00045198709, -0.00053757021, -0.00055030429, -0.00059555607, -0.00082048553, -0.0010504171, -0.0010410631, -0.0013833113, -0.0013222355, -0.0012391648, -0.0010352267, -0.00082861786, -0.0010668632, -0.0011791618, -0.0013303153, -0.0010150304, -0.00086158544, -0.0010423256, -0.001264332, -0.0011179706, -0.0012924796, -0.0013921244, -0.0015227843, -0.0011543537, -0.001143708, -0.0010282614, -0.00095421162, -0.0010284259, -0.0010431107, -0.0013277426, -0.0013104012, -0.0012658731, -0.0012899149, -0.0015310745, -0.0010605189, -0.00083136315, -0.0008910729, -0.00073506325, -0.00073203576, -0.00072533166, -0.00075146841, -0.00079362778, -0.00083394151, -0.0013402304, -0.0015848297, -0.0016454656, -0.0014946134, -0.0011887794, -0.0011142613, -0.0010003572, -0.00094841226, -0.0006311137, -0.00073567139, -0.0009140861, -0.0012110666, -0.001379972, -0.0015902289, -0.0016474437, -0.0016017357, -0.00094722329, -0.00072804661, -0.00049392756, -0.00036674475, -0.00039581515, -0.00053893263, -0.00095061161, -0.0012461236, -0.0012645866, -0.0014178089, -0.001351938, -0.0012540207, -0.00071471953, -0.00056387077, -0.00052584568, -0.00049032606, -0.00039897776, -0.00056738688, -0.0016403824, -0.0026869068, -0.001734404, -0.0013208098, -0.0012829111, -0.0013781109]}], 'scans': '/Users/smerdis/code/coupling/glm2/hemi_spm_full/workingdir/level1/_subject_id_MS/_fwhm_4/modelspec/swrsub-MS_ses-20150401_task-hemi_run-01_bold.nii'}, {'cond': [{'amplitudes': array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]), 'duration': [13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5], 'name': 'L', 'onset': [13.5, 40.5, 67.5, 94.5, 121.5, 148.5, 175.5, 202.5, 229.5, 256.5, 283.5]}, {'amplitudes': array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]), 'duration': [13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5, 13.5], 'name': 'R', 'onset': [27.0, 54.0, 81.0, 108.0, 135.0, 162.0, 189.0, 216.0, 243.0, 270.0, 297.0]}], 'hpf': 120.0, 'regress': [{'name': 'Realign1', 'val': [0.0, 0.0014878193, -0.00043815629, -0.0096104162, -0.010801428, -0.0030608597, -0.0069859294, -0.011192056, -0.0083012509, -0.015903831, -0.01010608, -0.013506098, -0.0081187638, -0.012766947, -0.013919015, -0.0026442932, -0.00088482696, -0.0045451212, -0.0048857706, -0.008736928, -0.007866851, -0.010290654, -0.01072081, 0.0027848777, 0.038145893, 0.028951541, -0.0055885461, -0.0074516519, -0.0026197024, -0.0073918952, 0.0033532827, 0.0016847644, -0.0060167285, -0.0073093377, -0.0074129983, -0.0077295991, -0.011392457, -0.0073903837, -0.0067032496, -0.011919441, -0.00015454296, 0.01153671, -0.0019333856, -0.0094218788, -0.016505574, -0.024340764, -0.017337441, -0.0094459992, -0.015125358, -0.016875232, -0.019561948, -0.016671375, -0.014622838, -0.01132506, -0.013148255, -0.014961655, -0.020094099, -0.023869088, -0.023556702, -0.020893799, -0.020730108, -0.016177188, -0.012526727, 0.0013606099, 0.0043493019, 0.00054057361, -0.0068409037, -0.008956376, -0.012175463, -0.011301785, -0.015862723, -0.024097914, -0.02004633, -0.020344902, -0.021569952, -0.021951376, 0.0076099529, -0.0014093007, -0.0049303301, -0.0091090392, -0.0062895712, -0.0021023028, -0.010068706, -0.010206617, -0.0055162551, -0.0057045863, -0.011746658, -0.011809102, -0.012052177, -0.007823017, -0.014063048, -0.019730138, -0.014153554, -0.0046806217, -0.0057233476, -0.021112661, -0.014212455, -0.014701647, -0.017546101, -0.012815062, -0.019283098, -0.01883671, -0.0077993295, -0.019811049, -0.02477754, -0.021556601, -0.024414903, -0.02329929, -0.018027585, -0.014913769, -0.01067844, -0.017657192, -0.016118311, -0.01229776, -0.023121438, -0.032806597, -0.02010335, -0.016063296, -0.021402899, -0.02395589, -0.024855887, -0.028319729, -0.014432378, -0.011903648, -0.013218661, -0.0078277104, -0.0058593566, -0.0054409836, -0.0044673408, -0.0013614207, -0.0056005124, -0.0069073868, -0.015971435, -0.01846955, -0.01887251, 0.012253525, 0.018822314, 0.011010345, 0.0055283992]}, {'name': 'Realign2', 'val': [0.0, 0.017564958, -0.028840727, 0.091407777, 0.4007414, -0.0019284972, 0.21442417, 0.076445189, 0.093557397, 0.30168907, 0.20274009, 0.66414546, 0.41355371, 0.025575106, -0.015183213, -0.12244574, 0.032095634, -0.036727545, -0.045754967, 0.050394637, 0.044965679, 0.32828126, 0.081167949, -0.026189856, -0.26260113, -0.11385234, 0.62936823, 0.071104879, 0.19219269, 0.13467534, 0.22515232, 0.0081804541, 0.057915144, 0.31959494, 0.32640571, 0.038017688, 0.43259542, 0.0358034, -0.05673072, 0.2848754, -0.12138346, 0.020761031, 0.25230302, 0.022401441, 0.22563223, 0.27979114, -0.032467121, -0.049995996, 0.20443836, 0.04618807, 0.10568107, 0.50852948, 0.040625044, -0.13470409, -0.092299871, 0.16845767, 0.029481641, 0.21732978, 0.00076997382, 0.19333181, 0.083409541, 0.2156781, 0.36734922, -0.11877246, -0.01780905, 0.01400425, 0.23334132, -0.11626374, 0.1544525, 0.14695289, 0.024208864, 0.33732774, -0.10647988, 0.28625099, 0.11479126, -0.12186258, -0.14857367, -0.090025585, -0.17978466, 0.12702684, -0.035578244, 0.17870209, -0.12140516, 0.071891439, 0.1099423, 0.090356764, 0.12379822, 0.05168499, 0.18354278, 0.086522853, -0.089700589, -0.027370632, 0.12143067, -0.22716717, -0.13002023, -0.12663228, -0.11081018, 0.19175656, 0.43158138, 0.05133387, -0.24809904, 0.064028281, -0.14567791, -0.16251153, -0.024021366, 0.3085532, -0.027892238, -0.10314166, 0.20411483, -0.16689212, 0.0055279154, -3.4122405e-05, 0.33641323, 0.16748175, -0.067793246, -0.12492655, -0.21850132, -0.092026513, -0.11870961, -0.086666888, -0.032216015, 0.12189581, -0.099322526, -0.053502268, 0.15544494, -2.8161526e-05, 0.25847835, -0.1964131, -0.0061997076, 0.084631128, -0.1028656, -0.15799837, -0.11853117, 0.19315382, 0.25996794, 0.3100668, 0.3502825, 0.31418219, -0.036570348]}, {'name': 'Realign3', 'val': [-1.7763568e-15, 0.077267344, 0.029858216, 0.080069017, 0.041460503, 0.0065826077, -0.020510178, -0.023005947, -0.020885416, -0.075442404, -0.051140931, -0.11501336, -0.1511262, -0.11600783, -0.10978206, -0.081365738, -0.098756636, -0.071008134, -0.0016755605, 0.076697268, 0.01643473, 0.030951967, -0.030732979, -0.051304705, -0.023641173, 0.10910504, 0.098982307, 0.043576379, -0.026900188, -0.016644801, -0.060239792, -0.061261044, -0.037356235, -0.0029201037, -0.081662944, -0.069338956, -0.037596493, -0.05061953, -0.053770506, -0.024106043, -0.11340537, -0.14858912, -0.0075109711, -0.021204862, -0.0058280666, -0.065328542, -0.068907922, -0.010047567, -0.060611338, -0.12714441, -0.059081008, -0.062301381, -0.10055671, -0.11112409, -0.05267131, -0.12795793, -0.13737416, -0.097442213, -0.061570809, -0.047434757, -0.055489492, -0.019128926, -0.054652846, -0.084140585, -0.026361635, 0.050842279, 0.06299146, 0.081835771, 0.111907, 0.057140649, 0.110778, 0.042700973, -0.017778875, -0.061476903, -0.070104491, -0.014296595, 0.058955444, 0.12118059, 0.10526156, 0.14918344, 0.15059624, 0.17274698, 0.0046380292, 0.0015218677, -0.094318108, -0.025325028, -0.046346359, -0.11494516, -0.095351494, -0.043919811, 0.075684277, 0.11672275, 0.13344975, 0.11132373, 0.1458994, 0.11917326, 0.14794625, 0.12823728, 0.063911921, 0.010113471, 0.11490038, 0.19118844, 0.15010463, 0.020715859, 0.091318695, 0.044151838, 0.026623549, 0.12636031, 0.16397525, -0.020048358, 0.036379591, 0.074299855, 0.079275871, 0.074587506, 0.046343024, 0.088080235, 0.10802378, 0.048840617, 0.015673073, 0.082618769, 0.090679132, 0.078367517, 0.10372221, 0.13958304, 0.21221953, 0.25068647, 0.21100316, 0.049871328, 0.039170328, 0.02560504, 0.048227714, 0.043508168, -0.011563781, 0.14026413, 0.018185043, 0.13366322, -0.0887535, 0.04893071, 0.079564016]}, {'name': 'Realign4', 'val': [0.0, 0.00082439428, 0.00024024293, 0.0012634534, 0.0014107348, 0.00085836219, 0.00044021174, -0.00010663773, -0.00010625345, -0.00044112528, 0.00025245049, 0.00045824838, -0.00096657055, -6.7749974e-05, -0.0023394759, -0.0017592055, -0.0014636161, -0.00076705379, 0.00021286284, 0.0010556474, 0.00097953038, 0.001017294, 0.00055470836, -0.0013168391, -0.0048543816, -0.0041678488, -0.0012970389, -0.0005522622, -0.00063215639, -0.00022406897, -0.0010538846, -0.00053928306, 0.00011894888, -0.00017638813, -0.00098258399, -0.00040959743, -0.00034849667, 0.00076952647, -0.00044232643, -0.00066371189, -0.001152738, -0.00086234446, -0.00015139899, -8.8250852e-05, 0.00017203024, 3.2216343e-06, -0.0016390742, -0.0009164178, -0.00089272134, -0.0010726497, -0.00012081319, -0.00066282022, -0.00085531749, -0.0014411453, -0.00063534653, -0.00046923001, -0.00027256329, -0.00074642431, -9.7571136e-05, -0.00014664432, -0.00021055589, 0.00045969508, -0.00056721164, -0.0019148193, -0.00055949843, -2.3918814e-05, 0.00021664178, 0.000840997, 0.00097368364, 0.00030719455, 0.0011187691, -0.00019912881, -4.4335846e-05, 9.5281009e-05, -0.00032432172, 0.00023275934, -0.0006821354, -0.00040691016, -0.00099627731, 8.6484561e-05, 0.00048960955, 0.00061433505, -0.00076935109, -0.00015697111, -0.001089035, 0.00027038394, 0.00054628603, -0.0012089767, 0.00011969935, -0.00042147847, -0.00021433111, 0.00072520327, 0.00062371786, 0.00050039262, 1.2718784e-05, 0.00085093453, 0.0004920642, 0.0004440925, -0.00011326714, -0.00040304924, 0.0015332651, 0.0023530747, 0.0013307957, -0.0022227378, -0.00071582232, -0.00017956776, 0.00018281895, 0.0015522907, 0.0010520959, -0.0025133097, -0.0028198392, -0.00090642328, -0.00030616201, -0.0010450423, -0.0010417042, 6.055099e-05, -0.00033524585, -0.0021158528, -0.0019991981, -0.00081110664, -0.00055595387, 7.8018366e-05, 0.0010687411, 0.0016704116, 0.0023302004, 0.0024204939, 0.0010014245, 7.0311102e-06, -0.00047630944, -0.00053705159, -0.0013676502, -0.00063933802, -0.00092854977, -0.0017631454, -0.0027390102, 3.5560776e-05, 0.00023910373, -0.00098627426, -0.0021425142]}, {'name': 'Realign5', 'val': [0.0, -0.0011482676, -0.0012885641, -0.0012172959, -0.00068057045, -0.00055029012, -0.00074141746, -0.0001827478, 0.00032705647, 0.00095393457, 0.00076486875, 0.00065846701, 0.000324086, 0.0014056582, 0.0012620597, 0.0011638413, 0.0013956605, 0.00069315226, 0.00076380516, 0.0012241216, 0.0017367497, 0.0009720239, 0.0012666201, -0.00020296501, 0.00064620045, -0.00012465589, -0.0003085229, 0.00087255562, 0.00069384646, 0.00100192, 0.0014757797, 0.0017023325, 0.0017003851, 0.0020937319, 0.0020839457, 0.0021892117, 0.0028090952, 0.0029298591, 0.0028549937, 0.0017209941, 0.001050397, 0.00062759688, 0.00055341469, 0.00079693169, 0.00098980312, 0.0010707552, 0.0014796129, 0.0017800001, 0.0011856474, 0.0013540832, 0.001086616, 0.0012174101, 0.0015812521, 0.0013341462, 0.00029347097, 7.2203494e-05, 0.00010901724, 0.0002591606, 9.3826175e-05, -0.00010931842, 0.00029953842, 0.00034914271, 0.00057510656, 0.00088270495, 0.00050340052, -0.000341436, -0.00092157958, -0.00084561637, -0.00050182247, -5.1414854e-06, 0.00039492848, 0.00092491634, 0.0012434996, 0.00051688168, 0.00064791361, 0.00060216318, -0.00019156655, -0.0008835557, -0.0014478526, -0.0013248799, -0.00068280416, -0.00061818921, 0.00044244154, 0.00098823594, 0.00076882749, 0.00063908438, 0.00033053808, 0.00016956946, 9.4314147e-05, 0.00012712997, -0.00031951097, -0.00089047991, 0.00010497587, 0.00016669634, 0.00014485778, -0.00087966275, -0.00063984008, -0.00084970937, -0.00098790467, -0.0010944756, -0.00053304135, -0.00035259185, -1.7628961e-05, -0.00043883127, -0.00070957574, -0.0011640465, -0.00060244298, -0.00031949052, -0.00070080464, -0.0010768939, -0.0015245412, -0.0014234292, -0.001588779, -0.0018216348, -0.001977734, -0.0016922238, -0.0014234896, -0.0011497778, -0.0012746239, -0.00089631798, -0.0010838538, -0.0006334304, -8.2418369e-05, 0.00053710374, 0.00127986, 0.0017396054, 0.0012714691, 0.0014816303, 0.0013108268, 0.0011755259, 0.0010697407, 0.0010702698, 0.0023775728, 0.00062012304, 0.0016329306, -0.00016380911, 0.00012388902, 0.00047383369, 0.00097151472]}, {'name': 'Realign6', 'val': [0.0, 3.1421969e-05, 1.3638414e-05, 0.00011170457, -0.00011330715, -0.0003402171, -4.723892e-06, 0.00052131141, 0.00065024023, 0.00067354535, 0.00086555317, 0.0010717482, 0.00072559016, 0.00063021339, 0.00054110783, 0.00012706563, 3.1785477e-05, 0.00033685445, 0.00049175426, 0.0010828953, 0.0012259207, 0.00098932363, 0.0010862697, 0.00065952954, -0.00099049454, -0.001335335, -2.5328008e-05, 0.00023384681, 0.00024235238, 0.00015973505, 0.00048251163, 0.00096784474, 0.00082002819, 0.0011079611, 0.0012034378, 0.001102421, 0.0007214974, 0.00025765828, 0.00071952117, 0.00055690229, -0.00017981644, -0.00054223574, 0.00020391048, 0.0011168229, 0.0014071337, 0.0015812495, 0.0017617069, 0.0018121071, 0.00099738423, 0.00038965897, 0.00044841073, 0.00029829047, 0.0002375195, 0.00028256085, 0.00066886876, 0.0010881263, 0.0010112362, 0.0011839785, 0.0013202454, 0.0010520904, 0.00079172535, 0.0001411081, 3.7552345e-05, 0.00016091359, 0.0001062545, 0.00022347539, 0.00055177481, 0.0012212655, 0.0011231186, 0.0011149445, 0.0013776272, 0.0011439045, 0.00065618139, 4.6367886e-05, 0.00010312867, 0.00019021691, -0.00020257048, 0.0002068287, 0.00074809396, 0.0011007745, 0.00098201585, 0.0010893411, 0.0010157436, 0.001175736, 0.00066551178, 0.00029609291, 0.00030664931, 0.00021405454, 2.5258073e-06, 0.00044077461, 0.0006901664, 0.0016022059, 0.0012916635, 0.0011433561, 0.0014224673, 0.0015398132, 0.00093777607, 0.00035155882, 0.00048372239, 0.0005689818, 0.00064496939, 0.00064500576, 0.0009106094, 0.0012289255, 0.0015820356, 0.001701336, 0.0018331912, 0.0017991303, 0.0011176148, -6.0750442e-05, 3.2557566e-06, 7.7297965e-05, 0.00019687463, 0.00023866646, 0.0007639932, 0.0014944208, 0.0013013522, 0.0013546396, 0.0015106278, 0.0014103679, 0.0010720424, 0.00037734311, 0.00051140356, 0.00013855994, 0.00013556118, 0.00026601137, 0.0009322083, 0.0017295334, 0.0016375681, 0.0015435804, 0.0019704322, 0.0018846838, 0.0017503445, 0.00099599463, 0.00019545268, 0.00062271254, -0.00033707234, -0.00024470728, 0.00024055402]}, {'name': 'Outlier1', 'val': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, {'name': 'Outlier2', 'val': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, {'name': 'Outlier3', 'val': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}], 'scans': '/Users/smerdis/code/coupling/glm2/hemi_spm_full/workingdir/level1/_subject_id_MS/_fwhm_4/modelspec/swrsub-MS_ses-20150401_task-hemi_run-02_bold.nii'}]

In [21]:
for cond in session_info:
    cond['regress'] = []
    cond['scans'] = ['/Users/smerdis/code/coupling/glm2/hemi_spm_full/workingdir/level1/_subject_id_MS/_fwhm_4/modelspec/swrsub-MS_ses-20150401_task-hemi_run-01_bold.nii']
session_info

[{'cond': [{'amplitudes': array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]),
    'duration': [13.5,
     13.5,
     13.5,
     13.5,
     13.5,
     13.5,
     13.5,
     13.5,
     13.5,
     13.5,
     13.5],
    'name': 'L',
    'onset': [13.5,
     40.5,
     67.5,
     94.5,
     121.5,
     148.5,
     175.5,
     202.5,
     229.5,
     256.5,
     283.5]},
   {'amplitudes': array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]),
    'duration': [13.5,
     13.5,
     13.5,
     13.5,
     13.5,
     13.5,
     13.5,
     13.5,
     13.5,
     13.5,
     13.5],
    'name': 'R',
    'onset': [27.0,
     54.0,
     81.0,
     108.0,
     135.0,
     162.0,
     189.0,
     216.0,
     243.0,
     270.0,
     297.0]}],
  'hpf': 120.0,
  'regress': [],
  'scans': ['/Users/smerdis/code/coupling/glm2/hemi_spm_full/workingdir/level1/_subject_id_MS/_fwhm_4/modelspec/swrsub-MS_ses-20150401_task-hemi_run-01_bold.nii']},
 {'cond': [{'amplitudes': array([ 1.,  1.,  1., 

In [22]:
level1design = spm.Level1Design()
level1design.inputs.timing_units = 'secs'
level1design.inputs.interscan_interval = 2.5
level1design.inputs.bases = {'hrf':{'derivs': [0,0]}}
level1design.inputs.session_info = session_info
level1design.run()

TypeError: 'NoneType' object is not iterable

In [40]:
model.SpecifyModel?

In [43]:
import nipype.interfaces.spm as spm

In [46]:
spm.Level1Design??